<h1 align="center"><font color="yellow">PGVector: Crie, armazene e consulte Embeddings OpenAI no PostgreSQL usando pgvector</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Vamos seguir os seguintes passos:

* docker pull ankane/pgvector

* docker-compose up -d

* Connect to the server 





In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy
from langchain.docstore.document import Document

import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


embeddings = OpenAIEmbeddings()

connection_string = PGVector.connection_string_from_db_params(
    driver=os.environ.get("DB_DRIVER", "psycopg"),
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD')
)


In [9]:
loader = TextLoader('./data/RESUMINHO_estado_da_Uniao.txt')
documents = loader.load()

In [10]:
documents

[Document(page_content='Senhora Presidente, Senhora Vice-Presidente, nossa Primeira Dama e Segundo Cavalheiro. Membros do Congresso e do Gabinete. Ministros do Supremo Tribunal Federal. Meus companheiros americanos.\n\nNo ano passado, a COVID-19 nos separou. Este ano finalmente estamos juntos novamente.\n\nHoje à noite, nos encontramos como democratas, republicanos e independentes. Mas o mais importante como americanos.\n\nCom um dever para com o outro, para com o povo americano, para com a Constituição.\n\nE com uma determinação inabalável de que a liberdade sempre triunfará sobre a tirania.\n\nSeis dias atrás, o russo Vladimir Putin procurou abalar as fundações do mundo livre pensando que poderia fazê-lo se curvar a seus modos ameaçadores. Mas ele calculou mal.\n\nEle pensou que poderia entrar na Ucrânia e o mundo viraria. Em vez disso, ele encontrou uma parede de força que nunca imaginou.\n\nEle conheceu o povo ucraniano.\n\nDo presidente Zelenskyy a todos os ucranianos, seu destemo

In [11]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
print(len(documents))
print(len(docs))

1
7


In [12]:
docs

[Document(page_content='Senhora Presidente, Senhora Vice-Presidente, nossa Primeira Dama e Segundo Cavalheiro. Membros do Congresso e do Gabinete. Ministros do Supremo Tribunal Federal. Meus companheiros americanos.\n\nNo ano passado, a COVID-19 nos separou. Este ano finalmente estamos juntos novamente.\n\nHoje à noite, nos encontramos como democratas, republicanos e independentes. Mas o mais importante como americanos.\n\nCom um dever para com o outro, para com o povo americano, para com a Constituição.\n\nE com uma determinação inabalável de que a liberdade sempre triunfará sobre a tirania.\n\nSeis dias atrás, o russo Vladimir Putin procurou abalar as fundações do mundo livre pensando que poderia fazê-lo se curvar a seus modos ameaçadores. Mas ele calculou mal.\n\nEle pensou que poderia entrar na Ucrânia e o mundo viraria. Em vez disso, ele encontrou uma parede de força que nunca imaginou.\n\nEle conheceu o povo ucraniano.', metadata={'source': './data/RESUMINHO_estado_da_Uniao.txt'}

In [13]:
from typing import List, Tuple

collection_name = 'Eddy_RESUMINHO_estado_da_Uniao'
# O Módulo PGVector tentará criar uma tabela com o nome da coleção (collection). 
# Portanto, certifique-se de que o nome da coleção seja exclusivo e que o usuário tenha permissão para criar uma tabela.
db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
)


In [19]:
query = "O que falou Putin?"
docs_with_score: List[Tuple[Document, float]] = db.similarity_search_with_score(query,
                                                                                k=2)


In [20]:
docs_with_score

[(Document(page_content='Os Estados Unidos são membros junto com outras 29 nações.\n\nImporta. A diplomacia americana é importante. americanos resolvem os problemas.\n\nO último ataque de Putin à Ucrânia foi premeditado e não provocado.\n\nEle rejeitou repetidos esforços de diplomacia.\n\nEle pensou que o Ocidente e a OTAN não responderiam. E ele pensou que poderia nos dividir em casa. Putin estava errado. Estávamos prontos. aqui está o que nós fizemos.\n\nNós nos preparamos extensivamente e com cuidado.\n\nPassamos meses construindo uma coalizão de outras nações amantes da liberdade da Europa e das Américas à Ásia e África para confrontar Putin.\n\nPassei incontáveis horas unificando nossos aliados europeus. Compartilhamos com o mundo com antecedência o que sabíamos que Putin estava planejando e exatamente como ele tentaria justificar falsamente sua agressão.\n\nContrariamos as mentiras da Rússia com a verdade.\n\nE agora que ele agiu, o mundo livre o está responsabilizando.', metadat

In [23]:
for doc, score in docs_with_score:
    print("-" * 100)
    print("Score: ", score)
    print("Conteúdo: ", doc.page_content)
    print("")
    print(doc.metadata)
    print("-" * 100)
    

----------------------------------------------------------------------------------------------------
Score:  0.16348958441733807
Conteúdo:  Os Estados Unidos são membros junto com outras 29 nações.

Importa. A diplomacia americana é importante. americanos resolvem os problemas.

O último ataque de Putin à Ucrânia foi premeditado e não provocado.

Ele rejeitou repetidos esforços de diplomacia.

Ele pensou que o Ocidente e a OTAN não responderiam. E ele pensou que poderia nos dividir em casa. Putin estava errado. Estávamos prontos. aqui está o que nós fizemos.

Nós nos preparamos extensivamente e com cuidado.

Passamos meses construindo uma coalizão de outras nações amantes da liberdade da Europa e das Américas à Ásia e África para confrontar Putin.

Passei incontáveis horas unificando nossos aliados europeus. Compartilhamos com o mundo com antecedência o que sabíamos que Putin estava planejando e exatamente como ele tentaria justificar falsamente sua agressão.

Contrariamos as mentiras 

In [24]:
store = PGVector(
    connection_string=connection_string, 
    embedding_function=embeddings, 
    collection_name='Eddy_RESUMINHO_estado_da_Uniao',
    distance_strategy=DistanceStrategy.COSINE
)

retriever = store.as_retriever(search_kwargs={"k": 3})


In [25]:
retriever

VectorStoreRetriever(tags=['PGVector', 'OpenAIEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.pgvector.PGVector object at 0x7f2d2e247e50>, search_type='similarity', search_kwargs={'k': 3})

In [26]:

retriever.get_relevant_documents(query='O que falou Putin?')


[Document(page_content='Os Estados Unidos são membros junto com outras 29 nações.\n\nImporta. A diplomacia americana é importante. americanos resolvem os problemas.\n\nO último ataque de Putin à Ucrânia foi premeditado e não provocado.\n\nEle rejeitou repetidos esforços de diplomacia.\n\nEle pensou que o Ocidente e a OTAN não responderiam. E ele pensou que poderia nos dividir em casa. Putin estava errado. Estávamos prontos. aqui está o que nós fizemos.\n\nNós nos preparamos extensivamente e com cuidado.\n\nPassamos meses construindo uma coalizão de outras nações amantes da liberdade da Europa e das Américas à Ásia e África para confrontar Putin.\n\nPassei incontáveis horas unificando nossos aliados europeus. Compartilhamos com o mundo com antecedência o que sabíamos que Putin estava planejando e exatamente como ele tentaria justificar falsamente sua agressão.\n\nContrariamos as mentiras da Rússia com a verdade.\n\nE agora que ele agiu, o mundo livre o está responsabilizando.', metadata